In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as prep
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import precision_recall_fscore_support
import librosa

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
files = pd.read_csv('/content/drive/MyDrive/mlsp/Audio_Classification-MLSP/annotations.csv')
label = files['label']

In [49]:
tmp = [];
test = [];
tmp = os.listdir('/content/drive/MyDrive/test/Audio_Classification-MLSP-test/')
files_names_test = tmp
for i in tmp:
    file_name = '/content/drive/MyDrive/test/Audio_Classification-MLSP-test/' + i;
    test.append(np.load(file_name));

In [5]:
ans = []
for i in files['fname']:
    file_name = '/content/drive/MyDrive/mlsp/Audio_Classification-MLSP/train/' + i;
    ans.append(np.load(file_name));

In [6]:
def mfcc(ans) :

    sample_rate = sr = 44100;

    ans = librosa.feature.mfcc(y=ans, sr=sample_rate, n_fft=2048, hop_length=512, n_mfcc=128, window="hann")
    ans = ans / np.max(np.abs(ans))

    return ans

In [7]:
for i in range(len(ans)):
    ans[i] = np.concatenate((ans[i], np.zeros(shape = (1, 128, 2584 - ans[i].shape[2]))), axis = 2)
    ans[i] = ans[i].flatten();
    ans[i] = mfcc(ans[i]);
    ans[i] = ans[i].reshape(128, 647, 1);

In [8]:
for i in range(len(test)):
  test[i] = np.concatenate((test[i], np.zeros(shape = (1, 128, 2584 - test[i].shape[2]))), axis = 2)
  test[i] = test[i].flatten();
  test[i] = mfcc(test[i]);  
  test[i] = test[i].reshape(128, 647, 1);

In [9]:
label = pd.get_dummies(label)
label.head()

,Bark,Crying_and_sobbing,Doorbell,Knock,Meow,Microwave_oven,Shatter,Siren,Vehicle_horn_and_car_horn_and_honking,Walk_and_footsteps
0,1,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0


In [10]:
ans = np.asarray(ans, dtype=np.float32)
ans.shape
test = np.asarray(test, dtype=np.float32)
test.shape

(201, 128, 647, 1)

In [11]:
# train_x, test_x, train_y, test_y = train_test_split(ans, label, test_size = 0.20, random_state = 333)
train_x = ans;
train_y = label;
test_x = test;

In [12]:
def details(true, pred):
  result = precision_recall_fscore_support(true, pred, average='weighted')
  print(print("Precision -> ", result[0] * 100, "\nRecall -> ", result[1] * 100, "\nF1 Score -> ", result[2] * 100));

In [13]:
train_x_flatten = [];
test_x_flatten = [];
train_y_flatten = [];
test_y_flatten = [];

for i in range (train_x.shape[0]):
    train_x_flatten.append(train_x[i].flatten())

for i in range (test_x.shape[0]):
    test_x_flatten.append(test_x[i].flatten())

train_y_flatten = train_y.idxmax(axis=1)
# test_y_flatten = test_y.idxmax(axis=1)

train_x_flatten = np.asarray(train_x_flatten)
test_x_flatten = np.asarray(test_x_flatten)

In [14]:
logistic_regression = LogisticRegression();
logistic_regression.fit(train_x_flatten, train_y_flatten);
# print(logistic_regression.score(test_x_flatten, test_y_flatten));

pred_logistic = logistic_regression.predict(test_x_flatten)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [15]:
# details(test_y_flatten, pred_logistic);

In [16]:
# DNN Model


# create model 
model = Sequential()
model.add(Dense(64, input_shape=(82816, ), activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(10, activation='softmax'))


# complie model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
train_x_dnn = [];

for i in range (train_x.shape[0]):
    tmp = train_x[i].flatten();
    train_x_dnn.append(tmp);
train_x_dnn[0].shape

train_x_dnn = np.asarray(train_x_dnn)

In [18]:
# DNN Model Fit
model.fit(train_x_dnn, train_y, epochs=20, batch_size=100)

Epoch 1/20
10/10 [==============================] - 3s 21ms/step - loss: 2.1771 - accuracy: 0.2180
Epoch 2/20
10/10 [==============================] - 0s 19ms/step - loss: 1.7747 - accuracy: 0.4190
Epoch 3/20
10/10 [==============================] - 0s 17ms/step - loss: 1.3533 - accuracy: 0.6140
Epoch 4/20
10/10 [==============================] - 0s 19ms/step - loss: 0.9542 - accuracy: 0.7650
Epoch 5/20
10/10 [==============================] - 0s 20ms/step - loss: 0.6482 - accuracy: 0.8720
Epoch 6/20
10/10 [==============================] - 0s 20ms/step - loss: 0.4356 - accuracy: 0.9220
Epoch 7/20
10/10 [==============================] - 0s 18ms/step - loss: 0.2901 - accuracy: 0.9600
Epoch 8/20
10/10 [==============================] - 0s 20ms/step - loss: 0.2029 - accuracy: 0.9720
Epoch 9/20
10/10 [==============================] - 0s 19ms/step - loss: 0.1435 - accuracy: 0.9830
Epoch 10/20
10/10 [==============================] - 0s 20ms/step - loss: 0.1006 - accuracy: 0.9880
Epoch 11/

In [41]:
# CNN Model


# create model
model = Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 647, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.25))
model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(10, activation='softmax'))


# compile model
opt = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [42]:
#CNN Model Fit
model.fit(train_x, train_y, epochs=50, batch_size=64, verbose=1)

Epoch 1/50
16/16 [==============================] - 4s 210ms/step - loss: 2.3003 - accuracy: 0.0920
Epoch 2/50
16/16 [==============================] - 3s 210ms/step - loss: 2.2818 - accuracy: 0.1630
Epoch 3/50
16/16 [==============================] - 3s 215ms/step - loss: 2.2466 - accuracy: 0.1960
Epoch 4/50
16/16 [==============================] - 3s 214ms/step - loss: 2.2145 - accuracy: 0.2180
Epoch 5/50
16/16 [==============================] - 3s 209ms/step - loss: 2.1260 - accuracy: 0.2860
Epoch 6/50
16/16 [==============================] - 3s 209ms/step - loss: 1.9298 - accuracy: 0.3800
Epoch 7/50
16/16 [==============================] - 3s 208ms/step - loss: 1.7333 - accuracy: 0.4230
Epoch 8/50
16/16 [==============================] - 3s 210ms/step - loss: 1.6360 - accuracy: 0.4610
Epoch 9/50
16/16 [==============================] - 3s 210ms/step - loss: 1.4826 - accuracy: 0.5110
Epoch 10/50
16/16 [==============================] - 3s 210ms/step - loss: 1.3640 - accuracy: 0.5910

In [43]:
# CNN Accuracy
# acc = model.evaluate(test_x, test_y)
# print(acc[1] * 100)

In [44]:
# CNN Predict
pred_cnn = model.predict(test)

In [45]:
pred = []
# true = []

# for index, row in test_y.iterrows():
#     for j in test_y.columns:
#         if row[j] == 1:
#             true.append(j);
#             break;
cnt = 0
for i in range(201):
    mx = ind = 0;
    for j in range(10):
        if pred_cnn[i][j] > mx :
            mx = pred_cnn[i][j];
            ind = j;
    pred.append(label.columns[ind])
    cnt = cnt + 1
    

pred = np.asarray(pred, dtype=str)
# true = np.asarray(true, dtype=str)

In [46]:
# details(true, pred)

In [65]:
prediction2 = pd.DataFrame(files_names_test, columns=['file_name'])

In [62]:
prediction2 = pd.DataFrame(pred, columns=['prediction'])

In [66]:
prediction2['prediction'] = pred

In [69]:
prediction2.to_csv('prediction.csv')

In [64]:
prediction2.head()

,prediction,field
0,Microwave_oven,MLSP_test1451403.npy
1,Microwave_oven,MLSP_test1443477.npy
2,Microwave_oven,MLSP_test140846.npy
3,Microwave_oven,MLSP_test1379148.npy
4,Microwave_oven,MLSP_test1404304.npy
